<a href="https://colab.research.google.com/github/mapbefun/mapbefun/blob/main/%EB%8F%84%EB%B4%89%EA%B5%AC%EB%AF%BC%EC%9C%A0%ED%98%95%ED%99%94_2016_K_prototype_method_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/datalab/drive")

In [ ]:
import sys, os
sys.path.append(os.pardir)
sys.path.append("/datalab/drive/MyDrive/dataset/graduate")
my_path="/datalab/drive/MyDrive/dataset/graduate"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/datalab/drive/MyDrive/dataset/graduate/dobong_2016.csv") 
df.columns

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
#사용하지 않는 컬럼 삭제
#분석에 불필요한 변수는 제거(가구일련번호는 단순히 가구원끼리 구분하기 위한 ID에 불가하고 데이터에 규칙성도 없으므로 삭제, 생활SOC시설과 관련성이 없는 설문 항목 삭제)
df = df.drop(columns = ['가구일련번호', '주로 이용하는 교통수단_평일', '주로 이용하는 교통수단_주말','주차 공간', '도봉구 영역별 만족도_주민참여','도봉구 영역별 만족도_지역경제','도봉구 영역별 만족도_민원행정서비스','도봉구 영역별 만족도_교육환경', '도봉구 영역별 만족도_복지서비스','도봉구 영역별 만족도_보건의료서비스','도봉구 영역별 만족도_교통/주차분야','도봉구 영역별 만족도_공원녹지' , '도봉구 영역별 만족도_문화/체육환경'  ])
df.columns

In [ ]:
df.count()

In [ ]:
df.describe()

In [ ]:
#변수명 변경  
df = df.rename(columns={"전체 가구원수" : "Ha_M_count" , "공공 편의시설 확대 순위_1순위" :  "Prefer_1" , "공공 편의시설 확대 순위_2순위" : "Perfer_2" , 
                        "주로 이용하는 교통수단_평일" : "Transport_weekdays" , "주로 이용하는 교통수단_주말" : "Transport_weekend" , "주차 공간" : "Parking_space" , 
                        "가구소득"  : "Ha_income" , "성별" : "Sex" , "연령별" : "Age" , "학력별" : "Edu" , "주택형태별" : "House_type" , "권역분류별" : "Zone" , 
                        "가구주와의 관계" : "Ha_relation" , "혼인상태" : "Marriage" ,  "도봉구 영역별 만족도_민원행정서비스" : "Satisfaction_admin" , 
                        "도봉구 영역별 만족도_교육환경" : "Satisfaction_edu" , "도봉구 영역별 만족도_문화/체육환경" :  "Satisfaction_culturesports" , 
                        "도봉구 영역별 만족도_복지서비스" : "Satisfaction_welfare" , "도봉구 영역별 만족도_보건의료서비스" : "Satisfaction_health" , 
                        "도봉구 영역별 만족도_교통/주차분야" : "Satisfaction_ traffic" , "도봉구 영역별 만족도_공원녹지" : "Satisfaction_park"})   
df.columns

In [ ]:
df.head(10)

In [ ]:
#결측치 확인
df.isnull().sum()   

In [ ]:
df.loc[df['Perfer_2'] != df['Perfer_2'], 'Perfer_2'] =12 #무응답인 경우 기타 코드값인 12번으로 대체함
df.head(10)

In [ ]:
df.isnull().sum()  #결측치 확인. 결측치가 모두 메꿔짐 확인.

In [ ]:
df = df.astype(int) #소수점 형식은 정수형으로 모두 변환
df

In [ ]:
df.mean()

K-Prototype 클러스터링

In [ ]:
pip install sklearn

In [ ]:
pip install kmodes

In [ ]:
pip install --upgrade kmodes

In [ ]:
pip install 'umap-learn==0.3.10'

In [ ]:
import os
import json

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime 
from tqdm import tqdm
from sklearn.preprocessing import PowerTransformer
import umap.umap_ as umap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import stats


from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes

import sklearn as sk


In [ ]:
#수치형 데이터에만 표준화 처리
numerical = df.iloc[:,[0, 3, 5, 6]]

for c in numerical.columns:
    pt = PowerTransformer()
    numerical.loc[:, c] =  pt.fit_transform(np.array(numerical[c]).reshape(-1, 1))

In [ ]:
#범주형 데이터는 따로 지정해서 군집 분석 시 수치형과 다른 기준으로 분석이 진행될 수 있도록 해준다.  
categorical = df.iloc[:,[1, 2, 4, 7, 8, 9, 10]]
categorical = pd.get_dummies(categorical) 

In [ ]:
# 범주형 열의 백분율은 나중에 임베딩에서 가중치 매개변수로 사용됨.
categorical_weight = len(df.select_dtypes(include='object').columns) / df.shape[1]

#Embedding numerical & categorical
fit1 = umap.UMAP(metric='euclidean').fit(numerical)
fit2 = umap.UMAP(metric='dice').fit(categorical)

In [ ]:
categorical_weight

In [ ]:
#Augmenting the numerical embedding with categorical
intersection = umap.umap_.general_simplicial_set_intersection(fit1.graph_, fit2.graph_, weight=categorical_weight)
intersection = umap.umap_.reset_local_connectivity(intersection)
embedding = umap.umap_.simplicial_set_embedding(fit1._raw_data, intersection, fit1.n_components, 
                                                fit1._initial_alpha, fit1._a, fit1._b, 
                                                fit1.repulsion_strength, fit1.negative_sample_rate, 
                                                200, 'random', np.random, fit1.metric, 
                                                fit1._metric_kwds, False)

plt.figure(figsize=(20, 10))
plt.scatter(*embedding.T, s=2, cmap='Spectral', alpha=1.0)
plt.show()

K-prototypes 적용

In [ ]:
# Choose optimal K using Elbow method(최적의 k개수 도출)
cost = []
for cluster in range(1, 12):
    try:
        kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
        kprototype.fit_predict(kprot_data_df, categorical = categorical_columns)
        cost.append(kprototype.cost_)
        print('Cluster initiation: {}'.format(cluster))
    except:
        break
# Import module for data visualization
from plotnine import *
import plotnine
# Converting the results into a dataframe and plotting them
df_cost = pd.DataFrame({'Cluster':range(1, 12), 'Cost':cost})

# Data viz
plotnine.options.figure_size = (8, 4.8)
(
    ggplot(data = df_cost)+
    geom_line(aes(x = 'Cluster',
                  y = 'Cost'))+
    geom_point(aes(x = 'Cluster',
                   y = 'Cost'))+
    geom_label(aes(x = 'Cluster',
                   y = 'Cost',
                   label = 'Cluster'),
               size = 10,
               nudge_y = 1000) +
    labs(title = 'Optimal number of cluster with Elbow Method')+
    xlab('Number of Clusters k')+
    ylab('Cost')+
    theme_minimal()
)

In [ ]:
kprot_data = df.copy()
for c in df.select_dtypes(exclude='object').columns:
    pt = PowerTransformer()
    kprot_data[c] =  pt.fit_transform(np.array(kprot_data[c]).reshape(-1, 1))

In [ ]:
categorical_columns = [1, 2, 4, 7, 8, 9, 10]

In [ ]:
#군집분석 플랜B
kproto = KPrototypes(n_clusters= 6, init='Cao', n_jobs = 4)
clusters = kproto.fit_predict(kprot_data, categorical=categorical_columns)

In [ ]:
clusters_df = pd.DataFrame(clusters)
clusters_df

In [ ]:
#군집화한 클러스트 개수를 출력
pd.Series(clusters).value_counts()

In [ ]:
all_df = pd.concat([df,clusters_df], sort=False).reset_index(drop=True)
all_df 

In [ ]:
#시각화
fig, ax = plt.subplots()
fig.set_size_inches((20, 10))
scatter = ax.scatter(embedding[:, 0], embedding[:, 1], s=2, c=clusters, cmap='tab20b', alpha=1.0)

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(num=15),
                    loc="lower left", title="Classes")
ax.add_artist(legend1)

In [ ]:
# Fit the cluster
kprototype = KPrototypes(n_jobs = -1, n_clusters = 10, init = 'Huang', random_state = 0)
kprototype.fit_predict(test_X, categorical = categorical_columns)

In [ ]:
# Cluster centorid
kprototype.cluster_centroids_
# Check the iteration of the clusters created
kprototype.n_iter_
# Check the cost of the clusters created
kprototype.cost_

In [ ]:
clf_kp = LGBMClassifier(colsample_by_tree=0.8)
cv_scores_kp = cross_val_score(clf_kp, lgbm_data, proto_clusters, scoring='f1_weighted')
print(f'CV F1 score for K-Prototypes clusters is {np.mean(cv_scores_kp)}')

In [ ]:
# 데이터 정규화(평균=0, 분산=1) 하기
from sklearn import preprocessing
train_scaled = preprocessing.scale(train_X)
train_scaled

In [ ]:
clf_kp.fit(lgbm_data, proto_clusters)
explainer_kp = shap.TreeExplainer(clf_kp)
shap_values_kp = explainer_kp.shap_values(HA_0)
shap.summary_plot(shap_values_kp, lgbm_data, plot_type="bar", plot_size=(15, 10))

In [ ]:
cluster_mean = cluster_mean.T
cluster_mean

In [ ]:
cluster_mean[1:].plot(figsize=(12,10), kind="barh" , subplots=True, layout=(1, 13) , sharey=True)